# Import data and libraries

In [1]:
from rfgap import RFGAP
import numpy as np

#Data
from aeon.datasets import load_gunpoint
from aeon.datasets import load_basic_motions

#Adjust the path
import sys
sys.path.insert(0, '/yunity/arusty/PF-GAP')

In [2]:
# Load Data
XTRAIN, YTRAIN = load_gunpoint(split="TRAIN")
XTEST, YTEST = load_gunpoint(split="TEST")

# XTRAIN, YTRAIN = load_basic_motions(split="TRAIN")
# XTEST, YTEST = load_basic_motions(split="TEST")

#Generate random static variables to test algorithm
XSTATIC = np.random.rand(XTRAIN.shape[0], 45)
XSTATIC_TEST = np.random.rand(XTEST.shape[0], 45)


# Basic Model Tests 
To ensure they are loaded correctly

## Testing RF_GAP seperately

In [3]:
prediction_type = 'classification'

rf = RFGAP(prediction_type = prediction_type)
rf.fit(np.random.rand(5, 5), np.random.randint(0, 2, size=(5,)))

print(rf.get_proximities().todense())

[[0.         0.34555556 0.03111111 0.52111111 0.10222222]
 [0.15509259 0.         0.35       0.09490741 0.4       ]
 [0.21607143 0.26130952 0.         0.23690476 0.28571429]
 [0.44607843 0.34803922 0.06666667 0.         0.13921569]
 [0.18585859 0.32777778 0.3030303  0.18333333 0.        ]]


## Testing the Rockets
Safety first, right?

In [4]:
from aeon.transformations.collection.convolution_based import MultiRocket, MiniRocket
from aeon.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train")
X_test, y_test = load_unit_test(split="test")
rocket = MultiRocket(n_kernels=512)
rocket.fit(X_train)

X_train = rocket.transform(X_train)
X_test = rocket.transform(X_test)

# RFGAP ROCKET

In [ ]:
from RFGAP_Rocket import RFGAP_Rocket

#NOT STATIC 
rf_rocket = RFGAP_Rocket(prediction_type = prediction_type, rocket = "Multi",
                         n_kernels=512) # Rocket Kwargs
rf_rocket.fit(XTRAIN, YTRAIN)
prox = rf_rocket.get_proximities().todense()

print(f"Proximities shape {prox.shape}")
print(f"Data shape {XTRAIN.shape}")
print(f"Proximities: \n{prox}")

Proximities shape (50, 50)
Data shape (50, 1, 150)
Proximities: 
[[0.         0.03662241 0.01714286 ... 0.04790072 0.00571429 0.0655244 ]
 [0.03828157 0.         0.00129032 ... 0.03793211 0.         0.04519644]
 [0.01266607 0.01326851 0.         ... 0.0200107  0.02591458 0.01454362]
 ...
 [0.03923773 0.03595286 0.00591209 ... 0.         0.07362637 0.04111988]
 [0.01248909 0.01771647 0.03155337 ... 0.01655373 0.         0.02626012]
 [0.04231982 0.04329365 0.         ... 0.0469057  0.         0.        ]]


In [11]:
from RFGAP_Rocket import RFGAP_Rocket

rf_rocket = RFGAP_Rocket(prediction_type = prediction_type, rocket = "Multi",
                         n_kernels=512) # Rocket Kwargs
rf_rocket.fit(XTRAIN, YTRAIN, XSTATIC)
prox_static = rf_rocket.get_proximities().todense()

print(f"Proximities shape {prox.shape}")
print(f"Data shape {XTRAIN.shape}")
print(f"Proximities: \n{prox}")

Proximities shape (50, 50)
Data shape (50, 1, 150)
Proximities: 
[[0.         0.03662241 0.01714286 ... 0.04790072 0.00571429 0.0655244 ]
 [0.03828157 0.         0.00129032 ... 0.03793211 0.         0.04519644]
 [0.01266607 0.01326851 0.         ... 0.0200107  0.02591458 0.01454362]
 ...
 [0.03923773 0.03595286 0.00591209 ... 0.         0.07362637 0.04111988]
 [0.01248909 0.01771647 0.03155337 ... 0.01655373 0.         0.02626012]
 [0.04231982 0.04329365 0.         ... 0.0469057  0.         0.        ]]


In [12]:
prox_static == prox

matrix([[ True, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        ...,
        [False, False, False, ...,  True, False, False],
        [False, False, False, ..., False,  True, False],
        [False, False,  True, ..., False,  True,  True]])